<h1>Nltk</h1>

O Nltk é uma biblioteca para processamento de texto natural em python. Com ela é possível realizar classificação de textos, treinar chunkers, entre outros

In [3]:
# -*- coding: utf-8 -*-

import nltk

Primeiramente, uma função bastante útil para o processamento de textos em português

In [4]:
from unicodedata import normalize

def normalize_text(text, to_lower=True):
    text = normalize('NFKD', text).encode('ASCII', 'ignore')

    final_text = text

    if to_lower:
        final_text = text.lower()

    return final_text

In [10]:
print(normalize_text('OlÁ, tudo bem? como você vai?'))
print(normalize_text('Olá, tudo bem? como você vai?', to_lower=False))

b'ola, tudo bem? como voce vai?'
b'Ola, tudo bem? como voce vai?'


Ao realizar a normalização de textos, considere excluir casos como o verbo de ligação "é" e o símbolo de parágrafo &para;